In [78]:
from langgraph.graph import StateGraph, START,END
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from typing import TypedDict
from pydantic import BaseModel, Field

In [79]:
load_dotenv()

True

In [80]:
import os

In [81]:
model = ChatGroq(
    model="llama-3.1-8b-instant",
    groq_api_key = os.getenv("Groq_api")
)

In [82]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description='Detailed feedback for the essay')
    score: int = Field(description='Score out of 10',ge=0,le=10)

In [83]:
Structured_model = model.with_structured_output(EvaluationSchema)

In [84]:
essay = """
Role of India in Artificial Intelligence

Artificial Intelligence (AI) is becoming one of the most important technologies in the world, and India is playing a growing role in this global transformation. With its large population, strong IT industry, and talented youth, India has great potential to become a leader in AI.

First, India has a huge number of skilled engineers, programmers, and data scientists. Every year, thousands of students graduate from engineering colleges and enter the tech industry. Many Indian professionals already work in top global companies and contribute to AI research and development. This gives India a strong foundation.

Second, the Indian government is actively supporting AI. Programs like Digital India, IndiaAI Mission, and National AI Strategy aim to use AI in healthcare, agriculture, education, and governance. For example, AI is being used to predict crop diseases, improve online learning, and make public services faster and more efficient.

Third, India has a large startup ecosystem. Many Indian startups are building AI solutions in fintech, health tech, edtech, and e-commerce. Cities like Bengaluru, Hyderabad, and Pune are becoming major AI hubs. These startups help create innovation, jobs, and new technologies for the country.

Finally, India is focusing on using AI for social good. Unlike many developed countries, India has unique challenges such as rural development, language diversity, and large healthcare demands. AI tools can help solve these problems by making technology more accessible to everyone.

In conclusion, India’s role in AI is increasing rapidly. With government support, skilled talent, and innovative startups, India is well-positioned to become a global leader in AI. If the country continues to invest in research and responsible AI development, it can shape the future of technology for both India and the world.
"""

In [85]:
prompt = f'Evaluate the language quality of the following essay and provide a feedbcak and assing a score out of 10 \n+ {essay}'

In [86]:
Structured_model.invoke(prompt)

BadRequestError: Error code: 400 - {'error': {'message': "tool call validation failed: parameters for tool EvaluationSchema did not match schema: errors: [missing properties: 'score']", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=EvaluationSchema> {"feedback": "The essay presents a well-structured and coherent argument about the role of India in Artificial Intelligence. It covers various aspects such as skilled talent, government support, startup ecosystem, and focus on social good. The writer effectively uses examples to illustrate the potential of India in AI. However, some sentences are wordy and could be improved for better clarity. Overall, the essay is well-written and demonstrates a good understanding of the topic. Score: 8/10"} </function>'}}

In [ ]:
from typing import TypedDict
class UPSCState(TypedDict):

    essay:str
    language_feedback:str
    analysis_feedback:str
    clarity_feedback:str
    overall_feedback:str
    individual_scores: Annotated[list[int],operator.add]
    avg_score:float

SyntaxError: invalid syntax (35125803.py, line 9)

In [ ]:
def evaluate_language(state:UPSCState):
    prompt = f'Evaluate the language quality of the following essay and provide a feedbcak and assing a score out of 10 \n+ {state["essay"]}'
    output = Structured_model.invoke(prompt)

    return {'language_feedback':output.feedback,'individual_scores':[output.score]}

In [ ]:
def evaluate_analysis(state:UPSCState):
    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedbcak and assing a score out of 10 \n+ {state["essay"]}'
    output = Structured_model.invoke(prompt)

    return {'analysis_feedback':output.feedback,'individual_scores':[output.score]}

In [ ]:
def evaluate_thought(state:UPSCState):
    prompt = f'Evaluate the clarity of though of the following essay and provide a feedbcak and assing a score out of 10 \n+ {state["essay"]}'
    output = Structured_model.invoke(prompt)

    return {'clarity_feedback':output.feedback,'individual_scores':[output.score]}

In [ ]:
def final_evaluation(state:UPSCState):
    prompt = f'Based on the following feedbacks create a summarized feedback \n+ language feedback - {state["language_feedback"]} \n+ depth of analysis feedback -{state["analysis_feedback"]} \n+ clarity of thought feedback - {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content

    # avg
    avg_score = sum(state['individual_score'])/len(state['individual_scores'])

    return {'overall_feedback':overall_feedbac,'avg_score':avg_score}

In [ ]:
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)

# edges
graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_thought')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')


graph.add_edge('final_evaluation', END)

workflow = graph.compile()



In [ ]:
workflow

In [ ]:
intial_state = {
    'essay':essay
}

In [ ]:
workflow.invoke(intial_state)